In [345]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Описание данных и задачи
Предсказать получает ли человек больше суммы N или меньше.
- age: возраст, действительное.
- workclass: тип работодателя, категориальное. 
- final_weight: итоговый вес обьекта, действительное.
- education: уровень образования, категориальное.
- education-num: количество лет обучения, действительное.
- marital-status: семейное положение, категориальное.
- occupation: профессия, категориальное.
- relationship: тип семейных отношений, категориальное.
- race: раса, категориальное.
- sex: пол, категориальное.
- cap-gain: прирост капитала, действительное.
- cap-loss: потери капитала, действительное.
- hours-per-week: количество часов работы в неделю, действительное.
- native-country: страна, категориальное.

Целевая переменная: sum_N – уровень заработка, категориальный (бинарный) признак

#### Считаем csv файлы с тренировочной и тестовой выборкой и добавляем названия колонок

In [346]:
names = ['age', 'workclass', 'final_weight', 'education' , 'education-num', 
         'marital-status', 'occupation', 'relationship', 'race', 'sex', 'cap-gain', 
         'cap-loss', 'hours-per-week', 'native-country']
df_test = pd.read_csv("test_empty.txt", names=names)
names.append("sum_N")
df_train = pd.read_csv("data.txt", names=names)

#### Посмотрим наполнение датафреймов

In [347]:
df_train.head(5)

,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,cap-gain,cap-loss,hours-per-week,native-country,sum_N
0,39,State-gov,77516.0,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40,United-States,<=N
1,50,Self-emp-not-inc,83311.0,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13,United-States,<=N
2,38,Private,215646.0,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40,United-States,<=N
3,53,Private,234721.0,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40,United-States,<=N
4,28,Private,338409.0,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40,Cuba,<=N


In [348]:
df_test.head(5)

,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,cap-gain,cap-loss,hours-per-week,native-country
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States
3,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States
4,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States


#### Заменим значения таргета на бинарные, так как решаем задачу бинарной классификации

In [349]:
df_train['sum_N'] = df_train['sum_N'].replace({' <=N': 0, ' >N': 1})

In [350]:
df_train.head(5)

,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,cap-gain,cap-loss,hours-per-week,native-country,sum_N
0,39,State-gov,77516.0,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40,United-States,0.0
1,50,Self-emp-not-inc,83311.0,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13,United-States,0.0
2,38,Private,215646.0,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40,United-States,0.0
3,53,Private,234721.0,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40,United-States,0.0
4,28,Private,338409.0,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40,Cuba,0.0


# 2. Исследование данных
## а) Узнаём базовые сведения о тренировочном датасете

#### Сведения о размерах данных

In [351]:
df_train.shape

(32573, 15)

#### Информация о типах данных

In [352]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32573 non-null  object 
 1   workclass       32572 non-null  object 
 2   final_weight    32572 non-null  float64
 3   education       32572 non-null  object 
 4   education-num   32572 non-null  object 
 5   marital-status  32572 non-null  object 
 6   occupation      32572 non-null  object 
 7   relationship    32572 non-null  object 
 8   race            32572 non-null  object 
 9   sex             32572 non-null  object 
 10  cap-gain        32572 non-null  float64
 11  cap-loss        32572 non-null  float64
 12  hours-per-week  32572 non-null  object 
 13  native-country  32572 non-null  object 
 14  sum_N           32571 non-null  float64
dtypes: float64(4), object(11)
memory usage: 3.7+ MB


#### Также узнаем подробности о данных

In [353]:
df_train.describe()

,final_weight,cap-gain,cap-loss,sum_N
count,3.257200e+04,32572.000000,32572.000000,32571.000000
mean,7.099108e+12,1077.315578,87.226943,0.241043
std,1.281227e+15,7384.067167,402.917749,0.427723
min,-4.214100e+04,-1.000000,-444.000000,0.000000
25%,1.177890e+05,0.000000,0.000000,0.000000
50%,1.783485e+05,0.000000,0.000000,0.000000
75%,2.370065e+05,0.000000,0.000000,0.000000
max,2.312322e+17,99999.000000,4356.000000,1.000000


#### Как видно из сведений выше, "age", "education-num" и "hours-per-week" отнесён к object. Это необходимо изменить. Но сперва посмотрим подробнее на данные каждой колонки

In [354]:
df_train['age'].unique() 

array(['39', '50', '38', '53', '28', '37', '49', '52', '31', '42', '30',
       '23', '32', '40', '34', '25', '43', '54', '35', '59', '56', '19',
       '20', '45', '22', '48', '21', '24', '57', '44', '41', '29', '18',
       '47', '46', '36', '79', '27', '67', '33', '76', '17', '55', '61',
       '70', '64', '71', '68', '66', '51', '58', '26', '60', '90', '75',
       '65', '77', '62', '63', '80', '72', '74', '69', '73', '81', '78',
       '88', '82', '83', '84', '85', 'totally_legit_string', '86', '-321',
       '1', '87'], dtype=object)

In [355]:
df_train['hours-per-week'].unique() 

array([' 40', ' 13', ' 16', ' 45', ' 50', ' 80', ' 30', ' 35', ' 60',
       ' 20', ' 52', ' 44', ' 15', ' 25', ' 38', ' 43', ' 55', ' 48',
       ' 58', ' 32', ' 70', ' 2', ' 22', ' 56', ' 41', ' 28', ' 36',
       ' 24', ' 46', ' 42', ' 12', ' 65', ' 1', ' 10', ' 34', ' 75',
       ' 98', ' 33', ' 54', ' 8', ' 6', ' 64', ' 19', ' 18', ' 72', ' 5',
       ' 9', ' 47', ' 37', ' 21', ' 26', ' 14', ' 4', ' 59', ' 7', ' 99',
       ' 53', ' 39', ' 62', ' 57', ' 78', ' 90', ' 66', ' 11', ' 49',
       ' 84', ' 3', ' 17', ' 68', ' 27', ' 85', ' 31', ' 51', ' 77',
       ' 63', ' 23', ' 87', ' 88', ' 73', ' 89', ' 97', ' 94', ' 29',
       ' 96', ' 67', ' 82', ' 86', ' 91', ' 81', ' 76', nan, ' 92', ' 61',
       ' 74', ' 95', ' United-States', ' 444424124141242'], dtype=object)

In [356]:
df_test['education-num'].unique()

array([7.00000000e+00, 9.00000000e+00, 1.20000000e+01, 1.00000000e+01,
       6.00000000e+00, 1.50000000e+01, 4.00000000e+00, 1.30000000e+01,
       1.40000000e+01, 1.60000000e+01, 3.00000000e+00, 1.10000000e+01,
       5.00000000e+00, 8.00000000e+00, 2.00000000e+00, 1.00000000e+00,
                  nan, 1.02302103e+11])

#### Заменяем явно выпадающие значения медианой

In [357]:
df_train['age'] = pd.to_numeric(df_train['age'], errors='coerce')
median_age = df_train['age'].median()
df_train.loc[df_train['age'] < 0, 'age'] = median_age
df_train.loc[df_train['age'] > 150, 'age'] = median_age
df_train['age'] = df_train['age'].fillna(median_age)

In [358]:
df_train['hours-per-week'] = pd.to_numeric(df_train['hours-per-week'], errors='coerce')
median_hours = df_train['hours-per-week'].median()
df_train.loc[df_train['hours-per-week'] < 0, 'hours-per-week'] = median_hours
df_train.loc[df_train['hours-per-week'] > 150, 'hours-per-week'] = median_hours
df_train['hours-per-week'] = df_train['hours-per-week'].fillna(median_hours)

In [359]:
df_train['education-num'] = pd.to_numeric(df_train['education-num'], errors='coerce')
median_num = df_train['education-num'].median()
df_train.loc[df_train['education-num'] < 0, 'education-num'] = median_num
df_train.loc[df_train['education-num'] > 100, 'education-num'] = median_num
df_train['education-num'] = df_train['education-num'].fillna(median_num)

#### Посмотрим на подробности о данных. Найдено ещё одно выпадающее значение - age=1

In [360]:
df_train.describe()

,age,final_weight,education-num,cap-gain,cap-loss,hours-per-week,sum_N
count,32573.000000,3.257200e+04,32573.000000,32572.000000,32572.000000,32573.000000,32571.000000
mean,38.570104,7.099108e+12,10.080650,1077.315578,87.226943,40.437295,0.241043
std,13.653808,1.281227e+15,2.572247,7384.067167,402.917749,12.345157,0.427723
min,1.000000,-4.214100e+04,1.000000,-1.000000,-444.000000,1.000000,0.000000
25%,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783485e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370065e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,2.312322e+17,16.000000,99999.000000,4356.000000,99.000000,1.000000


#### Удалим строки где возраст меньше 17


In [361]:
df_train = df_train[df_train['age'] != 1]
len(df_train) # удалено 10 записей

32563

#### Убедимся в адекватности значений и проверим изменился ли тип данных у колонок

In [362]:
df_train.describe()

,age,final_weight,education-num,cap-gain,cap-loss,hours-per-week,sum_N
count,32563.000000,3.256200e+04,32563.000000,32562.000000,32562.000000,32563.000000,32561.000000
mean,38.581642,7.101289e+12,10.080674,1077.615718,87.287513,40.437429,0.240810
std,13.640019,1.281424e+15,2.572641,7385.181096,402.964788,12.347050,0.427581
min,17.000000,1.228500e+04,1.000000,-1.000000,-444.000000,1.000000,0.000000
25%,28.000000,1.178285e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370615e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,2.312322e+17,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [363]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32563 entries, 0 to 32572
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32563 non-null  float64
 1   workclass       32562 non-null  object 
 2   final_weight    32562 non-null  float64
 3   education       32562 non-null  object 
 4   education-num   32563 non-null  float64
 5   marital-status  32562 non-null  object 
 6   occupation      32562 non-null  object 
 7   relationship    32562 non-null  object 
 8   race            32562 non-null  object 
 9   sex             32562 non-null  object 
 10  cap-gain        32562 non-null  float64
 11  cap-loss        32562 non-null  float64
 12  hours-per-week  32563 non-null  float64
 13  native-country  32562 non-null  object 
 14  sum_N           32561 non-null  float64
dtypes: float64(7), object(8)
memory usage: 4.0+ MB


#### Узнаем сколько пустых значений в оставшихся колонках

In [364]:
df_train.isnull().sum()

age               0
workclass         1
final_weight      1
education         1
education-num     0
marital-status    1
occupation        1
relationship      1
race              1
sex               1
cap-gain          1
cap-loss          1
hours-per-week    0
native-country    1
sum_N             2
dtype: int64

#### Заменяем пропуски данных медианой в колонках с числовым типом данных 

In [365]:
df_train['sum_N'] = df_train['sum_N'].fillna(df_train['sum_N'].median())
df_train['cap-gain'] = df_train['cap-gain'].fillna(df_train['cap-gain'].median())
df_train['cap-loss'] = df_train['cap-loss'].fillna(df_train['cap-loss'].median())
df_train['final_weight'] = df_train['final_weight'].fillna(df_train['final_weight'].median())

In [366]:
df_train.isnull().sum()

age               0
workclass         1
final_weight      0
education         1
education-num     0
marital-status    1
occupation        1
relationship      1
race              1
sex               1
cap-gain          0
cap-loss          0
hours-per-week    0
native-country    1
sum_N             0
dtype: int64

#### Выделяем колонки с категориальным типом данных и заполняем пропуски модой

In [367]:
categorical_columns = df_train.select_dtypes(include=['object']).columns
categorical_columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [368]:
for col in categorical_columns:
        most_common_value = df_train[col].mode()[0]
        df_train.loc[df_train[col] == ' ?', col] = most_common_value
        df_train[col].fillna(most_common_value, inplace=True)

In [369]:
df_train.isnull().sum()

age               0
workclass         0
final_weight      0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
cap-gain          0
cap-loss          0
hours-per-week    0
native-country    0
sum_N             0
dtype: int64

In [370]:
len(df_train)

32563

#### Применяем one hot encoding для категориальных признаков

In [371]:
df_train = pd.concat([df_train, pd.get_dummies(df_train['workclass'], prefix="workclass"),
                      pd.get_dummies(df_train['education'], prefix="education"),
                      pd.get_dummies(df_train['marital-status'], prefix="marital-status"),
                      pd.get_dummies(df_train['occupation'], prefix="occupation"),
                      pd.get_dummies(df_train['relationship'], prefix="relationship"),
                      pd.get_dummies(df_train['race'], prefix="race"),
                      pd.get_dummies(df_train['sex'], prefix="sex"),
                      pd.get_dummies(df_train['native-country'], prefix="native-country")],
                     axis=1)

In [372]:
df_train.drop(['workclass', 'education', 'marital-status', 'occupation','relationship', 'race', 'sex', 'native-country'],
                axis=1, inplace=True)

In [373]:
df_train.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
age,32563.0,3.858164e+01,1.364002e+01,17.0,28.0,37.0,48.0,9.000000e+01
final_weight,32563.0,7.101071e+12,1.281404e+15,12285.0,117830.0,178363.0,237058.0,2.312322e+17
education-num,32563.0,1.008067e+01,2.572641e+00,1.0,9.0,10.0,12.0,1.600000e+01
cap-gain,32563.0,1.077583e+03,7.385070e+03,-1.0,0.0,0.0,0.0,9.999900e+04
cap-loss,32563.0,8.728483e+01,4.029589e+02,-444.0,0.0,0.0,0.0,4.356000e+03
...,...,...,...,...,...,...,...,...
native-country_ Thailand,32563.0,5.527746e-04,2.350502e-02,0.0,0.0,0.0,0.0,1.000000e+00
native-country_ Trinadad&Tobago,32563.0,5.834843e-04,2.414874e-02,0.0,0.0,0.0,0.0,1.000000e+00
native-country_ United-States,32563.0,9.137364e-01,2.807571e-01,0.0,1.0,1.0,1.0,1.000000e+00
native-country_ Vietnam,32563.0,2.057550e-03,4.531423e-02,0.0,0.0,0.0,0.0,1.000000e+00


## б) Узнаём базовые сведения о тестовом датасете

In [378]:
df_test.shape

(16283, 14)

In [379]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16283 entries, 0 to 16282
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             16283 non-null  object 
 1   workclass       16282 non-null  object 
 2   final_weight    16282 non-null  float64
 3   education       16282 non-null  object 
 4   education-num   16282 non-null  float64
 5   marital-status  16282 non-null  object 
 6   occupation      16282 non-null  object 
 7   relationship    16282 non-null  object 
 8   race            16282 non-null  object 
 9   sex             16282 non-null  object 
 10  cap-gain        16282 non-null  float64
 11  cap-loss        16282 non-null  float64
 12  hours-per-week  16282 non-null  float64
 13  native-country  16282 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.7+ MB


In [381]:
df_test.describe()

,final_weight,education-num,cap-gain,cap-loss,hours-per-week
count,1.628200e+04,1.628200e+04,16282.000000,16282.000000,1.628200e+04
mean,1.894227e+05,6.283151e+06,1081.839393,87.895836,-1.969437e+04
std,1.057246e+05,8.017347e+08,7583.707692,403.093144,2.518175e+06
min,-2.132141e+04,1.000000e+00,0.000000,0.000000,-3.213214e+08
25%,1.166768e+05,9.000000e+00,0.000000,0.000000,4.000000e+01
50%,1.778295e+05,1.000000e+01,0.000000,0.000000,4.000000e+01
75%,2.383840e+05,1.200000e+01,0.000000,0.000000,4.500000e+01
max,1.490400e+06,1.023021e+11,99999.000000,3770.000000,9.900000e+01


#### Как видно из сведений выше, "age"отнесён к object. Это необходимо изменить.

In [383]:
df_test['age'].unique() 

array(['25', '38', '28', '44', '18', '34', '29', '63', '24', '55', '65',
       '36', '26', '58', '48', '43', '20', '37', '40', '72', '45', '22',
       '23', '54', '32', '46', '56', '17', '39', '52', '21', '42', '33',
       '30', '47', '41', '19', '69', '50', '31', '59', '49', '51', '27',
       '57', '61', '64', '79', '73', '53', '77', '80', '62', '35', '68',
       '66', '75', '60', '67', '71', '70', '90', '81', '74', '78', '82',
       '83', '85', '76', '84', '89', '88', '87', 'fssafsa121321ewd1231e2',
       '-10'], dtype=object)

In [384]:
df_test['age'] = pd.to_numeric(df_test['age'], errors='coerce')
median_age = df_test['age'].median()
df_test.loc[df_test['age'] < 0, 'age'] = median_age
df_test.loc[df_test['age'] > 150, 'age'] = median_age
df_test['age'] = df_test['age'].fillna(median_age)

In [387]:
median_hours = df_test['hours-per-week'].median()
df_test.loc[df_test['hours-per-week'] < 0, 'hours-per-week'] = median_hours
df_test.loc[df_test['hours-per-week'] > 100, 'hours-per-week'] = median_hours
df_test['hours-per-week'] = df_test['hours-per-week'].fillna(median_hours)

In [389]:
median_num = df_test['education-num'].median()
df_test.loc[df_test['education-num'] < 0, 'education-num'] = median_num
df_test.loc[df_test['education-num'] > 100, 'education-num'] = median_num
df_test['education-num'] = df_test['education-num'].fillna(median_num)

In [396]:
categorical_columns = df_test.select_dtypes(include=['object']).columns
for col in categorical_columns:
        most_common_value = df_test[col].mode()[0]
        df_test.loc[df_test[col] == ' ?', col] = most_common_value
        df_test[col].fillna(most_common_value, inplace=True)
        
not_categorical_columns = df_test.select_dtypes(include=['float']).columns
for col in not_categorical_columns:
        most_common_value = df_test[col].median()
        df_test.loc[df_test[col] == ' ?', col] = most_common_value
        df_test[col].fillna(most_common_value, inplace=True)

In [397]:
df_test.describe()

,age,final_weight,education-num,cap-gain,cap-loss,hours-per-week
count,16283.000000,1.628300e+04,16283.000000,16283.000000,16283.000000,16283.000000
mean,38.767242,1.894220e+05,10.072898,1081.772953,87.890438,40.392188
std,13.848350,1.057214e+05,2.567388,7583.479541,403.081354,12.478567
min,17.000000,-2.132141e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.166965e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.778295e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.383840e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000
